<a href="https://colab.research.google.com/github/TairCohen/personal-nutritionist-agent/blob/tair/basic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [5]:
if IN_COLAB:
    !pip install -q langgraph
    !pip install -q --upgrade langchain-text-splitters langchain-community langgraph
    !pip install -q langchain-openai
    !pip install faiss-cpu>=1.7.4
    !pip install -q langchain

In [128]:
# in jupyter, run this cell to reload the graph_nodes module
# if you make changes to it
# and want to test them without restarting the kernel
import importlib
import graph_nodes
importlib.reload(graph_nodes)

<module 'graph_nodes' from '/Users/tairlax/Desktop/30_hours/personal-nutritionist-agent/graph_nodes.py'>

In [142]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from graph_nodes import State, assistant, upload_image, display_image, identify_food, get_calories

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("assistant", assistant)
workflow.add_node("upload_image", upload_image)
workflow.add_node("display_image", display_image)
workflow.add_node("identify_food", identify_food)
workflow.add_node("get_calories", get_calories)


# Set the entrypoint as conversation
workflow.add_edge(START, "assistant")
workflow.add_edge("assistant", "upload_image")
workflow.add_edge("upload_image", "display_image")
workflow.add_edge("display_image", "identify_food")
workflow.add_edge("identify_food", "get_calories")
workflow.add_edge("get_calories", END)


# Compile
memory = MemorySaver()
graph = workflow.compile(interrupt_before=["display_image"], checkpointer=memory)

In [138]:
# display(Image(graph.get_graph().draw_mermaid_png()))

In [143]:
config = {"configurable": {"thread_id": "1"}}
# input_message = HumanMessage(content="hi! I'm Lance")
# output = graph.invoke({"messages": [input_message]}, config)
output = graph.invoke({}, config)